In [1]:
# Install dependencies
!pip install torchaudio matplotlib ipywidgets

# %% [code]
import torch
import torchaudio
from audioseal import AudioSeal
from IPython.display import display, HTML, Audio
import matplotlib.pyplot as plt

In [2]:
# Custom visualization functions
def plot_waveform_and_specgram(waveform, sr, title="Audio"):
    waveform = waveform.numpy().squeeze()
    
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 2, 1)
    plt.plot(waveform)
    plt.title(f"{title} Waveform")
    
    plt.subplot(1, 2, 2)
    plt.specgram(waveform, Fs=sr)
    plt.title(f"{title} Spectrogram")
    plt.show()

In [3]:
# %% [code]
# Initialize models with proper error handling
try:
    generator = AudioSeal.load_generator("audioseal_wm_16bits")
    detector = AudioSeal.load_detector("audioseal_detector_16bits")
except Exception as e:
    print(f"Error loading models: {e}")
    raise

In [4]:
# %% [code]
# Generate test audio with correct dimensions [batch=1, channels=1, samples]
original_audio = torch.randn(1, 1, 16000)  # 1s @16kHz
sr = 16000

# Verify input shape
print("Original audio shape:", original_audio.shape)


Original audio shape: torch.Size([1, 1, 16000])


In [5]:
# %% [code]
# Embed 3 watermarks sequentially with visualization
messages = [torch.randint(0, 2, (1, 16)) for _ in range(3)]
watermarked_audio = original_audio.clone()
alpha = 0.25  # Reduced strength for multi-watermarking

for idx, msg in enumerate(messages):
    # --- Embedding Phase ---
    # Ensure proper 3D input: [batch=1, channels=1, samples]
    watermark = generator.get_watermark(
        watermarked_audio,
        sample_rate=sr,
        message=msg
    )
    watermarked_audio = watermarked_audio + alpha * watermark
    
    # --- Detection Phase ---
    print(f"\nAfter embedding watermark {idx+1}:")
    print("-" * 50)
    
    # Detect all existing watermarks
    for detect_idx in range(idx + 1):
        detector.message = messages[detect_idx]  # Reset detector state
        prob, decoded_msg = detector.detect_watermark(
            watermarked_audio, 
            sample_rate=sr
        )
        ber = (messages[detect_idx] != decoded_msg.round()).float().mean()
        print(f"Watermark {detect_idx+1}: BER={ber:.2f}, Detection Prob={prob:.2f}")


After embedding watermark 1:
--------------------------------------------------
Watermark 1: BER=0.50, Detection Prob=0.26

After embedding watermark 2:
--------------------------------------------------
Watermark 1: BER=0.56, Detection Prob=0.32
Watermark 2: BER=0.50, Detection Prob=0.32

After embedding watermark 3:
--------------------------------------------------
Watermark 1: BER=0.62, Detection Prob=0.43
Watermark 2: BER=0.56, Detection Prob=0.43
Watermark 3: BER=0.50, Detection Prob=0.43


In [6]:

# %% [code]
# Final visualization (outside loop)
print("\nFinal Results:")
print("=" * 50)


Final Results:


In [7]:
# Calculate SNR
noise = watermarked_audio - original_audio
snr = 10 * torch.log10(original_audio.pow(2).mean() / noise.pow(2).mean())
print(f"Final SNR: {snr:.2f} dB")


Final SNR: 32.43 dB


In [8]:
# Visualize audio
plot_waveform_and_specgram(watermarked_audio.squeeze(), sr, "Multi-Watermarked Audio")
display(Audio(watermarked_audio.squeeze().numpy(), rate=sr))

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.